In [ ]:
import panel as pn
import param

pn.extension('tabulator')

The `panel.ReactiveExpr` pane renders a Param `rx` object which represents a reactive expression, displaying both the widgets that are part of the expression and the final output of the expression. The position of the widgets relative to the output can be set, or widgets can be removed entirely. [See the Param documentation on `param.rx`](https://param.holoviz.org/user_guide/Reactive_Expressions.html) for details on using `rx`.

#### Parameters:

The basic parameters are:

* **`object`** (param.reactive): A `param.reactive` expression 

The more advanced parameters which give you more control are:
 
* **`default_layout`** (panel.layout.base.ListPanel): A layout like Column, Row, etc. or a GridBox.
* **`center`** (bool): Whether to center the output.
* **`show_widgets`** (bool): Whether to display the widget inputs.
* **`widget_layout`** (panel.layout.base.ListPanel): The layout object to display the widgets in.
* **`widget_location`** (str): The location of the widgets relative to the output of the reactive expression. One of  'left', 'right', 'top', 'bottom', 'top_left', 'top_right', 'bottom_left', 'bottom_right', 'left_top' (default), 'right_top',right_bottom'.

For more layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).
___

The `param.rx` API is a powerful tool for building declarative and reactive UIs and the `ReactiveExpr` pane allows turning these declarative expressions into viewable components that render both the (widget) inputs and the outputs of the expression:

In [ ]:
v1 = pn.widgets.FloatInput(value=35)
v2 = pn.widgets.FloatInput(value=7)

expr = v1.rx() + v2.rx()

pn.ReactiveExpr(expr)

When Panel is imported an `rx` expression will automatically be rendered using the `ReactiveExpr` object, i.e. displaying `expr` on its own or putting it in a layout is exactly equivalent to the version above.

In [ ]:
expr

Note that this is different from using the expression as a *reference*, i.e. passing it to another component for it to resolve the value of the expression dynamically:

In [ ]:
pn.Row(v1, '`+`', v2, '`=`', pn.pane.Str(expr))

The *reference* approach should generally be preferred as it is more declarative and explicit, allowing Panel to efficiently update the existing view(s) rather than completely re-rendering the output.

The `ReactiveExpr` pane is therefore primarily a way to quickly and interactively work with an expression (particularly in a notebook) or generate a quick UI for a data pipeline. Once you have built the expression and want to integrate it in a larger application it is generally recommended that you use it as a reference as to narrowly scope the precise updates you want to make.

Let us work through this in a slightly more complex example, and build an expression to dynamically load some data and sample N rows from it:

In [ ]:
import pandas as pd

dataset = pn.widgets.Select(name='Pick a dataset', options={
    'penguins': 'https://datasets.holoviz.org/penguins/v1/penguins.csv',
    'stocks': 'https://datasets.holoviz.org/stocks/v1/stocks.csv'
})
nrows = pn.widgets.IntSlider(value=5, start=0, end=20, name='N rows')

# Load the currently selected dataset and sample nrows from it
df_rx = pn.bind(pd.read_csv, dataset).rx().sample(n=nrows)

df_rx

Now that we have an expression that does what we want we can use it as a reference to reactive update the `value` of a `Tabulator` widget:

In [ ]:
table = pn.widgets.Tabulator(df_rx, page_size=5, pagination='remote')

table.style.bar(cmap='RdYlGn_r')

pn.Row(pn.Column(dataset, nrows), table)

However, particularly for complex expressions with tons of inputs it may still be useful to use the `ReactiveExpr` object to render all the widget inputs:

In [ ]:
pn.Row(pn.ReactiveExpr(df_rx).widgets, table)